# Compare simulated top-N mzML with the real one

This notebook loads an existing beer1pos data, runs it through the simulator and compares the output.

In [ ]:
import sys
sys.path.append('C:\\Users\\joewa\\Work\\git\\clms\\Simulator\\codes')

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import pylab as plt
import matplotlib.patches as mpatches

In [ ]:
from VMSfunctions.Chemicals import *
from VMSfunctions.Chromatograms import *
from VMSfunctions.MassSpec import *
from VMSfunctions.Controller import *
from VMSfunctions.Common import *
from VMSfunctions.DataGenerator import *
from VMSfunctions.TopNExperiment import *
from VMSfunctions.Roi import *
from VMSfunctions.PlotsForPaper import *

In [ ]:
set_log_level_info()

## Parameters

In [ ]:
base_dir = 'C:\\Users\\joewa\\University of Glasgow\\Vinny Davies - CLDS Metabolomics Project\\'
mzml_path = os.path.join(base_dir, 'Data\\multibeers_urine_data\\beers\\fragmentation')
file_name = 'Beer_multibeers_1_T10_POS.mzML'

experiment_name = 'beer1pos_mzml_compare'
experiment_out_dir = os.path.join(base_dir, 'Manuscript\\2.2. Top N Simulations\\mzML')

In [ ]:
# min_rt = 3*60
# max_rt = 21*60

In [ ]:
min_rt = 0
max_rt = 1441

In [ ]:
kde_min_ms1_intensity = 0 # min intensity to be selected for kdes
kde_min_ms2_intensity = 0

ROI extraction parameters

In [ ]:
roi_mz_tol = 10
roi_min_length = 1
roi_min_intensity = 0
roi_start_rt = min_rt
roi_stop_rt = max_rt

Top-N parameters

In [ ]:
isolation_window = 1   # the isolation window in Dalton around a selected precursor ion
ionisation_mode = POSITIVE
N = 10
rt_tol = 15
mz_tol = 10
min_ms1_intensity = 1.75E5 # minimum ms1 intensity to fragment

In [ ]:
mzml_out = os.path.join(experiment_out_dir, 'beer1pos_simulated.mzML')

## Check minimum ms1 intensity to fragment

In [ ]:
import pymzml
run = pymzml.run.Reader(os.path.join(mzml_path, file_name), obo_version='4.0.1',
                        MS1_Precision=5e-6,
                        extraAccessions=[('MS:1000016', ['value', 'unitName'])])

selected_precursor_intensities = []
for scan_no, scan in enumerate(run):
    precursors = scan.selected_precursors
    try:
        if len(precursors) > 0:
            selected_precursor_intensities.append(precursors[0]['i'])
    except KeyError:
        continue

In [ ]:
plt.boxplot(np.log(selected_precursor_intensities))

In [ ]:
np.min(selected_precursor_intensities)

In [ ]:
np.max(selected_precursor_intensities)

## Train densities

In [ ]:
ds = DataSource()
ds.load_data(mzml_path, file_name=file_name)
print('MS1')
ds.plot_data(file_name, ms_level=1, max_data=100000, min_rt=min_rt, max_rt=max_rt)
print('MS2')
ds.plot_data(file_name, ms_level=2, max_data=100000, min_rt=min_rt, max_rt=max_rt)

## Prepare dataset

Extract all ROIs

In [ ]:
densities = PeakDensityEstimator(kde_min_ms1_intensity, kde_min_ms2_intensity, min_rt, max_rt, plot=True)
densities.kde(ds, file_name, 2, bandwidth_mz_intensity_rt=1.0, bandwidth_n_peaks=1.0)
ps = PeakSampler(densities)

In [ ]:
mzml_file = os.path.join(mzml_path, file_name)
good_roi, junk = make_roi(mzml_file, mz_tol=roi_mz_tol, mz_units='ppm', min_length=roi_min_length,
                          min_intensity=roi_min_intensity, start_rt=roi_start_rt, stop_rt=roi_stop_rt)

In [ ]:
all_roi = good_roi + junk

How many singleton and non-singleton ROIs?

In [ ]:
len([roi for roi in all_roi if roi.n == 1])

In [ ]:
len([roi for roi in all_roi if roi.n > 1])

How many can be fragmented above the min_ms1_intensity threshold?

In [ ]:
min_ms1_intensity

In [ ]:
keep = []
for roi in all_roi:
    if np.count_nonzero(np.array(roi.intensity_list) > min_ms1_intensity) > 0:
        keep.append(roi)

print(len(keep))
all_roi = keep

In [ ]:
len(all_roi)

Turn ROIs into chromatograms/chemicals

In [ ]:
set_log_level_debug()
rtcc = RoiToChemicalCreator(ps, all_roi)

In [ ]:
data = rtcc.chemicals
save_obj(data, os.path.join(experiment_out_dir, 'dataset.p'))

## Test Single Top-N

In [ ]:
density = ps.density_estimator
set_log_level_warning()
pbar = True

In [ ]:
mass_spec = IndependentMassSpectrometer(ionisation_mode, data, density=density)
controller = TopNController(mass_spec, N, isolation_window, mz_tol,
                            rt_tol, min_ms1_intensity)
controller.run(min_rt, max_rt, pbar)

In [ ]:
controller.write_mzML('my_analysis', mzml_out)

### Check the following

- Number of frag events
- Number of scans
- How many matches

In [ ]:
def count_stuff(input_file, min_rt, max_rt):
    run = pymzml.run.Reader(input_file, MS1_Precision=5e-6,
                            extraAccessions=[('MS:1000016', ['value', 'unitName'])],
                            obo_version='4.0.1')
    mzs = []
    rts = []
    intensities = []
    count_ms1_scans = 0
    count_ms2_scans = 0
    cumsum_ms1_scans = []
    cumsum_ms2_scans = []    
    count_selected_precursors = 0
    for spectrum in run:
        ms_level  = spectrum['ms level']
        current_scan_rt, units = spectrum.scan_time
        if units == 'minute':
            current_scan_rt *= 60.0            
        if min_rt < current_scan_rt < max_rt:                    
            if ms_level == 1:
                count_ms1_scans += 1
                cumsum_ms1_scans.append((current_scan_rt, count_ms1_scans, ))
            elif ms_level == 2:
                try:
                    selected_precursors = spectrum.selected_precursors  
                    count_selected_precursors += len(selected_precursors)
                    mz = selected_precursors[0]['mz']
                    intensity = selected_precursors[0]['i']

                    count_ms2_scans += 1
                    mzs.append(mz)
                    rts.append(current_scan_rt)
                    intensities.append(intensity)
                    cumsum_ms2_scans.append((current_scan_rt, count_ms2_scans, ))                    
                except KeyError:
                    # print(selected_precursors)
                    pass
                    
    print('Number of ms1 scans =', count_ms1_scans)                
    print('Number of ms2 scans =', count_ms2_scans)
    print('Total scans =', count_ms1_scans + count_ms2_scans)    
    print('Number of selected precursors =', count_selected_precursors)
    return np.array(mzs), np.array(rts), np.array(intensities), np.array(cumsum_ms1_scans), np.array(cumsum_ms2_scans)

In [ ]:
def find_match(query, min_rts, max_rts, min_mzs, max_mzs, mz_list, rt_list, intensity_list):
    
    # check ranges
    query_mz, query_rt, query_intensity = query
    min_rt_check = min_rts <= query_rt
    max_rt_check = query_rt <= max_rts
    min_mz_check = min_mzs <= query_mz
    max_mz_check = query_mz <= max_mzs
    idx = np.nonzero(min_rt_check & max_rt_check & min_mz_check & max_mz_check)[0]
    
    # get mz, rt and intensity of matching indices
    matches_mz = mz_list[idx]
    matches_rt = rt_list[idx]
    matches_intensity = intensity_list[idx]

    if len(idx) == 0: # no match
        return None
    
    elif len(idx) == 1: # single match
        return (matches_mz[0], matches_rt[0], matches_intensity[0], )
    
    else: # multiple matches, take the closest in rt
        diffs = [np.abs(rt - query_rt) for rt in matches_rt]
        idx = np.argmin(diffs)
        return (matches_mz[idx], matches_rt[idx], matches_intensity[idx], )

    
def match(mz_list_1, rt_list_1, intensity_list_1, mz_list_2, rt_list_2, intensity_list_2, mz_tol, rt_tol):  
    
    if mz_tol is not None: # create mz range for matching in ppm
        min_mzs = np.array([mz * (1-mz_tol/1e6) for mz in mz_list_2])
        max_mzs = np.array([mz * (1+mz_tol/1e6) for mz in mz_list_2])
        
    else: # create mz ranges by rounding to 2dp
        min_mzs = np.around(mz_list_2, decimals=2)
        max_mzs = np.around(mz_list_2, decimals=2)
        mz_list_1 = np.around(mz_list_1, decimals=2)    
    
    # create rt ranges for matching
    min_rts = np.array([rt - rt_tol for rt in rt_list_2])
    max_rts = np.array([rt + rt_tol for rt in rt_list_2])
        
    matches = {}    
    for i in range(len(mz_list_1)): # loop over query and find a match
        query = (mz_list_1[i], rt_list_1[i], intensity_list_1[i], )
        match = find_match(query, min_rts, max_rts, min_mzs, max_mzs, mz_list_2, rt_list_2, intensity_list_2)
        matches[query] = match
    return matches

def check_found_matches(matches, left_label, right_label, N=20):
    found = [key for key in matches if matches[key] is not None]
    print('Found %d/%d (%f)' % (len(found), len(matches), len(found)/len(matches)))

    print('%s\t\t\t\t\t\t%s' % (left_label, right_label))
    for key, value in list(matches.items())[0:N]:
        if value is not None:
            print('mz %.2f rt %.4f intensity %.4f\tmz %.2f rt %.4f intensity %.4f' % (key[0], key[1], key[2], value[0], value[1], value[2]))

In [ ]:
simulated_input_file = mzml_out
simulated_mzs, simulated_rts, simulated_intensities, simulated_cumsum_ms1, simulated_cumsum_ms2 = count_stuff(
    simulated_input_file, min_rt, max_rt)

In [ ]:
real_input_file = 'C:\\Users\\joewa\\University of Glasgow\\Vinny Davies - CLDS Metabolomics Project\\Data\\multibeers_urine_data\\beers\\fragmentation\\Beer_multibeers_1_T10_POS.mzML'
real_mzs, real_rts, real_intensities, real_cumsum_ms1, real_cumsum_ms2 = count_stuff(real_input_file, min_rt, max_rt)

#### Plot number of scans

In [ ]:
plt.rcParams.update({'font.size': 14})    
plt.plot(real_cumsum_ms1[:, 0], real_cumsum_ms1[:, 1], 'r')
plt.plot(real_cumsum_ms2[:, 0], real_cumsum_ms2[:, 1], 'b')
plt.plot(simulated_cumsum_ms1[:, 0], simulated_cumsum_ms1[:, 1], 'r--')
plt.plot(simulated_cumsum_ms2[:, 0], simulated_cumsum_ms2[:, 1], 'b--')
plt.legend(['Actual MS1', 'Actual MS2', 'Simulated MS1', 'Simulated MS2'])
plt.xlabel('Retention Time (s)')
plt.ylabel('Cumulative sum')
# plt.title('Cumulative number of MS1 and MS2 scans', fontsize=18)
plt.rcParams.update({'font.size': 14})
plt.tight_layout()
plt.savefig('num_scans.png', dpi=300)

#### Check the number of matches

In [ ]:
mz_tol = None
rt_tol = 5
matches = match(real_mzs, real_rts, real_intensities, simulated_mzs, simulated_rts, simulated_intensities, mz_tol, rt_tol)
check_found_matches(matches, 'Real', 'Simulated')

In [ ]:
mz_tol = None
rt_tol = 10
matches = match(real_mzs, real_rts, real_intensities, simulated_mzs, simulated_rts, simulated_intensities, mz_tol, rt_tol)
check_found_matches(matches, 'Real', 'Simulated')

In [ ]:
mz_tol = None
rt_tol = 15
matches = match(real_mzs, real_rts, real_intensities, simulated_mzs, simulated_rts, simulated_intensities, mz_tol, rt_tol)
check_found_matches(matches, 'Real', 'Simulated')

#### Plot the matches

In [ ]:
unmatched_intensities = []
matched_intensities = []
for key, value in list(matches.items()):
    intensity = key[2]
    if value is None:
        unmatched_intensities.append(intensity)
    else:
        matched_intensities.append(intensity)

In [ ]:
plt.rcParams.update({'font.size': 18})    
plt.figure()
temp1 = plt.hist(np.log(matched_intensities), bins = np.linspace(10,20,50), color='blue')
temp2 = plt.hist(np.log(unmatched_intensities), bins = np.linspace(10,20,50), color='red')
# plt.title('Matched precursor intensities')
blue_patch = mpatches.Patch(color='blue', label='Matched')
red_patch = mpatches.Patch(color='red', label='Unmatched')
plt.legend(handles=[blue_patch, red_patch])
plt.xlabel('log(intensity)')
plt.ylabel('Precursor count')
plt.tight_layout()
plt.savefig('matched_intensities.png', dpi=300)

In [ ]:
def plot_matches(matches, min_mz, max_mz, min_rt, max_rt, out_file=None):
    plt.figure(figsize=(12, 6))
    plt.rcParams.update({'font.size': 24})    
    for key in matches:
        mz, rt, intensity = key    
        if min_mz < mz < max_mz and min_rt < rt < max_rt:
            if matches[key] is not None:
                plt.plot([rt], [mz], marker='.', markersize=5, color='blue', alpha=0.1)
            else:
                plt.plot([rt], [mz], marker='.', markersize=5, color='red', alpha=0.1)  

    blue_patch = mpatches.Patch(color='blue', label='Matched')
    red_patch = mpatches.Patch(color='red', label='Unmatched')
    plt.legend(handles=[blue_patch, red_patch])
#     plt.title('Matched fragmentation events', fontsize=30)
    plt.xlabel('Retention Time (s)')
    plt.ylabel('m/z')
    plt.tight_layout()
    if out_file is not None:
        plt.savefig(out_file, dpi=300)

In [ ]:
plot_matches(matches, 50, 1000, 180, 1260, out_file='matched_precursors.png')

#### Count the number of XCMS picked-peaks

In [ ]:
min_ms1_intensity = 0
rt_range = [(0, 1441)]
mz_range = [(0, 9999)]

In [ ]:
csv_file = 'C:\\Users\\joewa\\University of Glasgow\\Vinny Davies - CLDS Metabolomics Project\\Manuscript\\2.2. Top N Simulations\\mzML\\extracted_peaks_ms1.csv'
df = get_df(csv_file, min_ms1_intensity, rt_range, mz_range)

In [ ]:
count_df = df.groupby('filename').size().reset_index(name='counts')

In [ ]:
count_df

In [ ]:
make_boxplot(df, 'filename', 'mz', None, 
             'mz distributions of MS1 features detected by CentWave')

make_boxplot(df, 'filename', 'rt', None, 
             'rt distributions of MS1 features detected by CentWave')

make_boxplot(df, 'filename', 'log_intensity', None, 
             'Intensity distributions of MS1 features detected by CentWave')